# Sentiment Analysis across music genres
In this section we will perform sentiment analysis on the top songs of each genre. These genres are given in the *music_genre.txt* file, and in order to get the top songs and their lyrics we will use both the Spotify API through the *spotipy* package for searching and the Genius API through the *lyricsgenius* package to get lyrics.

In [ ]:
import lyricsgenius